In [2]:
import numpy as np
def file2matrix(filename):
    fr = open(filename)
    arrayOLines = fr.readlines()
    numOLines = len(arrayOLines)
    retMat = np.zeros((numOLines,3))
    labels = []
    index = 0
    for line in arrayOLines:
        line = line.strip()
        listFromLine = line.split('\t')
        renMat[index,:] = listFromLine[:3]
        labels.append(int(listFromLine[-1]))
        index += 1
    return renMat,labels
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    rangeVals = maxVals-minVals
    m = dataSet.shape[0]
    normDataSet = dataSet-np.tile(minVals,(m,1))
    normDataSet = normDataSet/np.tile(rangeVals,(m,1))
    return normDataSet,rangeVals,minVals
def kNN_classify(X,dataSet,labels,k):
    m = len(dataSet)
    diffMat = np.tile(X,(m,1))-dataSet
    sqare_diffMat = diffMat ** 2
    sum_sqare_diffMat = sqare_diffMat.sum(axis=1)
    distances = np.sqrt(sum_sqare_diffMat)
    soretedDistIndex = distances.argsort()
    classCounts={}
    for i in range(k):
        votelabel = labels[soretedDistIndex[i]]
        classCounts[votelabel]=classCounts.get(votelabel,0)+1
    sortedClassCount = sorted(classCounts.items(),key=lambda p : p[1],reverse=True)
    return sortedClassCount[0][0]

In [4]:
def datingClassTest():
    hoRatio = 0.1
    dataSet,labels = file2matrix('datingTestSet2.txt')
    normDataSet,rangeVals,minVals = autoNorm(dataSet)
    m = normDataSet.shape[0]
    numTestSet = int(m*hoRatio)
    errorCount =0.0
    trainX, trainLabel = normDataSet[numTestSet:,:],labels[numTestSet:]
    for i in range(numTestSet):
        classifyResult = kNN_classify(normDataSet[i,:],trainX,trainLabel,3)
        if classifyResult != labels[i]:
            errorCount += 1.0
    print('the total error rate is : %f' %(errorCount/float(numTestSet)))
def classifyPerson():
    reslutList = ['not at all','in small doses','in large doses']
    ffMiles = float(input('frequent flier miles earned per year'))
    percentTats = float(input('percentage of time spen playing video games'))
    iceCream = float(input('liters of ice cream consumed per year'))
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
    normMat, rangVals, minVals = autoNorm(datingDataMat)
    X = [[ffMiles,percentTats,iceCream]]
    normX = (X-minVals)/rangVals
    classifyResult = kNN_classify(normX,normMat,datingLabels,3)
    print('you will probably like this person:',reslutList[classifyResult-1])
# if __name__ == '__main__':
#     classifyPerson()

In [5]:
import numpy as np
import os
import time
def img2Vector(filename):
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        linsStr = fr.readline()
        for j in range(32):
            returnVect[0,i*32+j] = int(linsStr[j])
    return returnVect
def handWritingClassTest_by_theory():
    hwLabels = []
    trainFileList = os.listdir('trainingDigits')
    m = len(trainFileList)
    trainMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = trainFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        trainMat[i,:]=img2Vector('trainingDigits/%s'%(fileNameStr))
    testFlieList = os.listdir('testDigits')
    m_test = len(testFlieList)
    error_count = 0.0
    for i in range(m_test):
        fileNameStr = testFlieList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        testVec = img2Vector('testDigits/%s'%(fileNameStr))
        classifyResult = kNN_classify(testVec,trainMat,hwLabels,3)
        if classifyResult != classNumStr:
            error_count += 1.0
    print(error_count)
    print(error_count/float(m_test))


In [6]:
import numpy as np
import os
import time
from sklearn.neighbors import KNeighborsClassifier as KNN
def img2Vector(filename):
    returnVect = np.ones((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,i*32+j] = int(lineStr[j])
    return returnVect

In [8]:
def handWritingClassTest_by_sklearn():
    hwLabels = []
    trainFileList = os.listdir('trainingDigits')
    m = len(trainFileList)
    trainMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = trainFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        trainMat[i,:]= img2Vector('trainingDigits/%s'%(fileNameStr))
    neigh = KNN(n_neighbors=3,algorithm='auto')
    neigh.fit(trainMat,hwLabels)
    testFileList = os.listdir('testDigits')
    m_test = len(testFileList)
    error_count = 0.0
    for i in range(m_test):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        testVec = img2Vector('testDigits/%s'%(fileNameStr))
        classifyResult = neigh.predict(testVec)
        if classifyResult != classNumStr:
            error_count += 1.0
    print("\nthe total number of errors is: %d", error_count)
    print("\nthe total error rate is : %f" % (error_count/float(m_test)))  